In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import errno
import copy
from copy import deepcopy
import csv
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def preprocess_compasdataset(df):
    df = df[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text',
                 'sex', 'priors_count', 'days_b_screening_arrest', 'decile_score',
                 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

    # Indices of data samples to keep
    ix = df['days_b_screening_arrest'] <= 30
    ix = (df['days_b_screening_arrest'] >= -30) & ix
    ix = (df['is_recid'] != -1) & ix
    ix = (df['c_charge_degree'] != "O") & ix
    ix = (df['score_text'] != 'N/A') & ix
    df = df.loc[ix,:]
    df['length_of_stay'] = (pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in'])).apply(lambda x: x.days)

    # Restrict races to African-American and Caucasian
    df = df.loc[~df['race'].isin(['Native American','Hispanic','Asian','Other']),:]

    df = df[['sex','race','age_cat','c_charge_degree','score_text','priors_count','is_recid', 'two_year_recid','length_of_stay']]

    df['priors_count'] = df['priors_count'].apply(lambda x: 0 if x <= 0 else ('1 to 3' if 1 <= x <= 3 else 'More than 3'))
    df['length_of_stay'] = df['length_of_stay'].apply(lambda x: '<week' if x <= 7 else ('<3months' if 8 < x <= 93 else '>3months'))
    df['score_text'] = df['score_text'].apply(lambda x: 'MediumHigh' if (x == 'High')| (x == 'Medium') else x)
    df['age_cat'] = df['age_cat'].apply(lambda x: '25 to 45' if x == '25 - 45' else x)

    df['sex'] = df['sex'].replace({'Female': 1.0, 'Male': 0.0})
    df['race'] = df['race'].apply(lambda x: 1.0 if x == 'Caucasian' else 0.0)

    df = df[['two_year_recid', 'sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']]

    protected_attributes = ['sex', 'race']
    label_name = 'two_year_recid'
    categorical_features = ['age_cat', 'priors_count', 'c_charge_degree']
    features = categorical_features + [label_name] + protected_attributes

    # privileged classes
    privileged_classes = {"sex": [1.0], "race": [1.0]}

    # protected attribute maps
    protected_attribute_map = {"sex": {0.0: 'Male', 1.0: 'Female'},
                                "race": {1.0: 'Caucasian', 0.0: 'Not Caucasian'}}


    data = StandardDataset(df, label_name, favorable_classes=[0],
                           protected_attribute_names=protected_attributes,
                           privileged_classes=[privileged_classes[x] for x in protected_attributes],
                           categorical_features=categorical_features,
                           features_to_keep=features,
                           metadata={'label_maps': [{1.0: 'Did recid.', 0.0: 'No recid.'}],
                                     'protected_attribute_maps': [protected_attribute_map[x] for x in protected_attributes]})

    return data

In [3]:
############ Reweighing ##############

def reweighing_data(train, unprivileged_group, privileged_group):
    RW = Reweighing(unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
    RW.fit(train)
    train_transformed = RW.transform(train)
    
    train_set = train_transformed

    # change weights to whole numbers
    for i in range(train_transformed.instance_weights.size):
        train_transformed.instance_weights[i] = (round(train_transformed.instance_weights[i] / 0.1) * 0.1) * 10
        weights = copy.deepcopy(train_transformed.instance_weights)

    # change train_transformed.features and train_transformed.labels and train_transformed.protected_attributes according to the weights of each instance
    for i in range(train_transformed.features.shape[0]):
        row = copy.deepcopy(train_transformed.features[i])
        row_label = copy.deepcopy(train_transformed.labels[i])
        row_protected_attributes = copy.deepcopy(train_transformed.protected_attributes[i])
        row_protected_attributes.resize(1,2)
        row.resize(1,train_transformed.features.shape[1])
        row_label.resize(1,1)
        weight = int(weights[i])
        for j in range(weight-1):
            train_transformed.features = np.concatenate((train_transformed.features,row))
            train_transformed.labels = np.concatenate((train_transformed.labels,row_label))
            train_transformed.protected_attributes = np.concatenate((train_transformed.protected_attributes,row_protected_attributes))

    # change the train_transformed to a numpy array of ones to match number of rows in features
    train_transformed.instance_weights = np.ones(train_transformed.features.shape[0])

    print("reweighing complete\n")

    return train_set, train_transformed

In [4]:
def make_prediction(train, test, unprivileged_groups, privileged_groups):
    
    ################## adversarial debiasing #################

    sess = tf.Session()
    debiased_model_reweighing = AdversarialDebiasing(privileged_groups = privileged_groups,
                                                     unprivileged_groups = unprivileged_groups,
                                                     scope_name='debiased_classifier', debias=True, sess=sess)
    debiased_model_reweighing.fit(train)
    dataset_debiasing_test_reweighing = debiased_model_reweighing.predict(test)
    sess.close()
    tf.reset_default_graph()

    print("Adversarial Debiasing")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_debiasing_test_reweighing,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_debiasing_test_reweighing,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

    accuracy_adversarial = accuracy_score(y_true = test.labels, y_pred = dataset_debiasing_test_reweighing.labels)

    metrics_adversarial = [metric_test.mean_difference(),acc_test.disparate_impact(), acc_test.equal_opportunity_difference(), acc_test.average_odds_difference(), acc_test.theil_index()]


    ##################### prejudice remover #####################
    prejudice_model_reweighing = PrejudiceRemover(eta=100, sensitive_attr='sex')
    prejudice_model_reweighing.fit(train)
    dataset_prejudice_test_reweighing = prejudice_model_reweighing.predict(test)

    print("Prejudice Remover")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_prejudice_test_reweighing,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_prejudice = accuracy_score(y_true=test.labels, y_pred=dataset_prejudice_test_reweighing.labels)

    metrics_prejudice = [metric_test.mean_difference(), acc_test.disparate_impact(), 0.0, 0.0, acc_test.theil_index()]


    ##################### normal neural net #####################
    sess = tf.Session()
    neural_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                        unprivileged_groups = unprivileged_groups,
                                        scope_name='debiased_classifier', debias=False, sess=sess)
    neural_model.fit(train)
    dataset_neural_test = neural_model.predict(test)
    sess.close()
    tf.reset_default_graph()

    print("Non-debiasing")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_neural_test,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_neural_test,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_nondebiasing = accuracy_score(y_true=test.labels, y_pred=dataset_neural_test.labels)

    metrics_nondebiasing = [metric_test.mean_difference(),acc_test.disparate_impact(), acc_test.equal_opportunity_difference(), acc_test.average_odds_difference(), acc_test.theil_index()]


    ##################### ensemble #####################
    pred_labels_test = []
    for i in range(0, len(test.features)):
        arr_test = mode([dataset_debiasing_test_reweighing.labels[i], dataset_prejudice_test_reweighing.labels[i], dataset_neural_test.labels[i]])
        pred_labels_test.append(arr_test[0][0])
        dataset_ensemble_test = test.copy()
        dataset_ensemble_test.labels = np.array(pred_labels_test)

    print("Ensemble")

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_ensemble_test,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_ensemble_test,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_ensemble = accuracy_score(y_true=test.labels, y_pred=dataset_ensemble_test.labels)
    metrics_ensemble = [metric_test.mean_difference(),acc_test.disparate_impact(), acc_test.equal_opportunity_difference(), acc_test.average_odds_difference(), acc_test.theil_index()]

    accuracy_scores = [accuracy_adversarial, accuracy_prejudice, accuracy_nondebiasing, accuracy_ensemble]
    fairness_metrics = [metrics_adversarial, metrics_prejudice, metrics_nondebiasing, metrics_ensemble]
    print("compiling all metrics complete\n")
    
    return accuracy_scores, fairness_metrics

In [5]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

df = pd.read_csv('dataset/compas-scores-two-years.csv')
dataset_orig = preprocess_compasdataset(df)

train, test = dataset_orig.split([0.7], shuffle=True)

print("splitting data complete")

train_transformed, dataset_transf_train = reweighing_data(train, unprivileged_groups, privileged_groups)

print('  With reweighing')
accuracy_reweigh, fairness_metrics_reweigh =  make_prediction(dataset_transf_train, test, unprivileged_groups, privileged_groups)
metrics_adversarial_reweigh = fairness_metrics_reweigh[0]
metrics_prejudice_reweigh = fairness_metrics_reweigh[1]
metrics_nondebiasing_reweigh = fairness_metrics_reweigh[2]
metrics_ensemble_reweigh = fairness_metrics_reweigh[3]

print('  Without reweighing')
accuracy, fairness_metrics =  make_prediction(train_transformed, test, unprivileged_groups, privileged_groups)
metrics_adversarial = fairness_metrics[0]
metrics_prejudice = fairness_metrics[1]
metrics_nondebiasing = fairness_metrics[2]
metrics_ensemble = fairness_metrics[3]

print('prediction completed')

splitting data complete
reweighing complete

  With reweighing
Adversarial Debiasing
Prejudice Remover
Non-debiasing
Ensemble
compiling all metrics complete

  Without reweighing
Adversarial Debiasing
Prejudice Remover
Non-debiasing
Ensemble
compiling all metrics complete

prediction completed


In [6]:
from IPython.display import Markdown, display

display(Markdown("#### Accuracy scores and Fairness metrics with reweighing"))

columns = ['Adversarial', 'Prejudice', 'Nondebiased', 'Ensemble']
index = ['Accuracy', 'Mean difference', 'Disparate impact', 'Equal Opportunity Difference', 'Average Odds Difference', 'Theil Index']

mean_difference = [metrics_adversarial_reweigh[0], metrics_prejudice_reweigh[0], metrics_nondebiasing_reweigh[0], metrics_ensemble_reweigh[0]]
disparate_impact = [metrics_adversarial_reweigh[1], metrics_prejudice_reweigh[1], metrics_nondebiasing_reweigh[1], metrics_ensemble_reweigh[1]]
equal_opportunity_difference = [metrics_adversarial_reweigh[2], metrics_prejudice_reweigh[2], metrics_nondebiasing_reweigh[2], metrics_ensemble_reweigh[2]]
average_odds_difference = [metrics_adversarial_reweigh[3], metrics_prejudice_reweigh[3], metrics_nondebiasing_reweigh[3], metrics_ensemble_reweigh[3]]
theil_index = [metrics_adversarial_reweigh[4], metrics_prejudice_reweigh[4], metrics_nondebiasing_reweigh[4], metrics_ensemble_reweigh[4]]

data_reweighted = [accuracy_reweigh, mean_difference, disparate_impact, equal_opportunity_difference, average_odds_difference, theil_index]
df = pd.DataFrame(data_reweighted, index = index, columns=columns)
print(df)

display(Markdown("#### Accuracy scores and Fairness metrics without reweighing"))

mean_difference = [metrics_adversarial[0], metrics_prejudice[0], metrics_nondebiasing[0], metrics_ensemble[0]]
disparate_impact = [metrics_adversarial[1], metrics_prejudice[1], metrics_nondebiasing[1], metrics_ensemble[1]]
equal_opportunity_difference = [metrics_adversarial[2], metrics_prejudice[2], metrics_nondebiasing[2], metrics_ensemble[2]]
average_odds_difference = [metrics_adversarial[3], metrics_prejudice[3], metrics_nondebiasing[3], metrics_ensemble[3]]
theil_index = [metrics_adversarial[4], metrics_prejudice[4], metrics_nondebiasing[4], metrics_ensemble[4]]

data = [accuracy, mean_difference, disparate_impact, equal_opportunity_difference, average_odds_difference, theil_index]
df = pd.DataFrame(data, index = index, columns=columns)
print(df)

#### Accuracy scores and Fairness metrics with reweighing

                              Adversarial  Prejudice  Nondebiased  Ensemble
Accuracy                         0.649621   0.636995     0.652146  0.652146
Mean difference                 -0.134439   0.046883    -0.115147 -0.115147
Disparate impact                 0.806432   1.083317     0.829473  0.829473
Equal Opportunity Difference    -0.041353   0.000000    -0.035828 -0.035828
Average Odds Difference         -0.119742   0.000000    -0.097749 -0.097749
Theil Index                      0.202573   0.203122     0.202986  0.202986


#### Accuracy scores and Fairness metrics without reweighing

                              Adversarial  Prejudice  Nondebiased  Ensemble
Accuracy                         0.648990   0.636995     0.645202  0.644571
Mean difference                 -0.158791   0.046883    -0.102212 -0.137582
Disparate impact                 0.771370   1.083317     0.844937  0.801908
Equal Opportunity Difference    -0.066710   0.000000    -0.023238 -0.050862
Average Odds Difference         -0.144103   0.000000    -0.086050 -0.122939
Theil Index                      0.213597   0.203122     0.210578  0.207207
